In [ ]:
!pip install emoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234926 sha256=2da6179ea295b87e688d7430eeec3f3f16dcf9318799a65e368e9a65516a9217
  Stored in directory: /root/.cache/pip/wheels/9a/b8/0f/f580817231cbf59f6ade9fd132ff60ada1de9f7dc85521f857
Successfully built emoji


In [ ]:
import os
import pandas as pd
import numpy as np
import re
import time
import emoji
import string
import nltk
nltk.download('omw-1.4')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords
from nltk.tokenize import WhitespaceTokenizer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

In [ ]:
for dirname, _, filenames in os.walk('/content/gdrive/My Drive/1019 Final Project/data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
train_df = pd.read_csv('/content/gdrive/My Drive/1019 Final Project/data/train.csv')
test_df = pd.read_csv('/content/gdrive/My Drive/1019 Final Project/data/test.csv')

In [ ]:
train_df

In [ ]:
train_df = train_df.drop_duplicates('text')
len(train_df)

7503

In [ ]:

def removeEmoji(text):
    return emoji.replace_emoji(text, '')

def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

def tweet_cleaner(text):
    text = text.lower() #convert to lowercase
    text = text.translate(str.maketrans("","", string.punctuation)) #remove punctuation
    text = removeEmoji(text) #remove emoji
    tk = WhitespaceTokenizer()  #tokenize text to list of words without space
    textsplit = tk.tokenize(text)
    lemmatizer = WordNetLemmatizer()
    tag = nltk.pos_tag(textsplit) #get tags (noun, verb...)

    result = []
    for i,w in enumerate(tag):
        word = w[0]
        tag = w[1]
        if word not in stopwords.words("english") and not word.startswith("http") and "\\" not in word and '#' not in word and '@' not in word:
            result.append(lemmatizer.lemmatize(word,get_wordnet_pos(tag))) #lemmartize word to current tense

        text = ' '.join(result) #join back the result
    return text

In [ ]:
result = []
for index, tweet in enumerate(train_df['text'].head(10)):
    tweet = tweet_cleaner(tweet)
    result.append(tweet)

In [ ]:
train_df['text'].map(tweet_cleaner)

In [ ]:
train_df.head(10)

,id,keyword,location,text,target
0,1,NaN,NaN,deed reason earthquake may allah forgive u,1
1,4,NaN,NaN,forest fire near la ronge sask canada,1
2,5,NaN,NaN,resident ask shelter place notify officer evac...,1
3,6,NaN,NaN,13000 people receive wildfire evacuation order...,1
4,7,NaN,NaN,get send photo ruby alaska smoke wildfire pour...,1
5,8,NaN,NaN,rockyfire update california hwy 20 close direc...,1
6,10,NaN,NaN,flood disaster heavy rain cause flash flooding...,1
7,13,NaN,NaN,im top hill see fire wood,1
8,14,NaN,NaN,there emergency evacuation happen building acr...,1
9,15,NaN,NaN,im afraid tornado come area,1


In [ ]:
def word_counter(words):
    counter = {}
    for word in words:
        if word in counter:
            counter[word] += 1
        else:
            counter[word] = 1
    return counter

In [ ]:
word_list = []
for line in result:
    word_freq = word_counter(line.split())
    word_list.append(word_freq)

In [ ]:
%load_ext Cython
%load_ext line_profiler

ModuleNotFoundError: ignored

In [ ]:
%%cython

import nltk
import string
import emoji
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer
import pandas as pd
import numpy as np
import time

start = time.time()

def removeEmoji(text):
    return emoji.replace_emoji(text, '')

def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

def tweet_cleaner(text):
    text = text.lower() #convert to lowercase
    text = text.translate(str.maketrans("","", string.punctuation)) #remove punctuation
    text = removeEmoji(text) #remove emoji
    tk = WhitespaceTokenizer()  #tokenize text to list of words without space
    textsplit = tk.tokenize(text)
    lemmatizer = WordNetLemmatizer()
    tag = nltk.pos_tag(textsplit) #get tags (noun, verb...)

    result = []
    for i,w in enumerate(tag):
        word = w[0]
        tag = w[1]
        if word not in stopwords.words("english") and not word.startswith("http") and "\\" not in word and '#' not in word and '@' not in word:
            result.append(lemmatizer.lemmatize(word,get_wordnet_pos(tag))) #lemmartize word to current tense

    text = ' '.join(result) #join back the result
    return text

train_df = pd.read_csv('train.csv')

result = []

for index, tweet in enumerate(train_df['text']):
    tweet = tweet_cleaner(tweet)
    result.append(tweet)

def word_counter(words):
    counter = {}
    for word in words:
        if word in counter:
            counter[word] += 1
        else:
            counter[word] = 1
    return counter

word_dict = []

for line in result:
    word_dict.append(word_counter(line.split()))

end = time.time()
print(end-start)

24.696743726730347
